Before we begin, let's get an overview of the CUDA driver version and the GPUs running on the server by executing the `nvidia-smi` command below. Highlight the cell below by clicking on it and then either hit `Ctrl+Enter` on the keyboard or click on the `Run` button on the toolbar above. The output will be visible below the cell.

In [ ]:
!nvidia-smi

# Learning Objectives

In this tutorial, the goal is to:
* Parallelize the single-GPU code using CUDA Memcpy and streams
* Understand intra-node topology and underlying technologies like GPUDirect P2P and their implication on program performance

# Multi-GPU Programming

In this section we first cover the principle behind decomposing data among the GPUs, known as domain decomposition. Then, we understand and implement the baseline multi-GPU code using `cudaSetDevice` and `cudaMemcpy` functions. 

### Domain Decomposition

Before we begin, we define two important terms:

* **Latency:** The amount of time it takes to take a unit of data from point A to point B. For example, if 4B of data can be transferred from point A to B in 4 $\mu$s, that is the latency of transfer.
* **Bandwidth:** The amount of data that can be transferred from point A to point B in a unit of time. For example, if the width of the bus is 64KiB and latency of transfer between point A and B is 4 $\mu$s, the bandwidth is 64KiB * (1/4$\mu$s) = 1.6 GiB/s.

To parallelize our application to multi-GPUs, we first review the different methods of domain decomposition available to us for splitting the data among the GPUs, thereby distributing the work. Broadly, we can divide data into either stripes or tiles.

* **Stripes**: They minimize the number of neighbours, require communication among less neighbours, and are optimal for latency bound communication.

* **Tiles**: They minimize surface area/ volume ratio of the grid, require communicating less data, and are optimal for bandwidth bound communication.

![domain_decomposition](../../images/domain_decomposition.png)

When we divide the global grid between GPUs, only the boundaries of each GPU-local grid need to be communicated with the neighboring GPUs, as they need the updated grid-point values for the next iteration. Therefore, we use horizontal stripes (as C/ C++ are row-major) in our tutorials for domain decomposition, enabling data parallelism.

### Halo Exchange

We term the exchange of top and bottom rows after each iterations the "halo exchange". Review the image below and notice that we update the topmost and bottomost rows of the grid to implement the periodic boundary condition. Recall that the left and right columns of the grid constitute Dirichlet boundary conditions (that is, constant value).

![halo_exchange](../../images/halo_exchange.png)

## CUDA concepts: Part 1

### Setting the GPU

To verify that our system has multiple GPUs in each node, run the command below:

In [ ]:
!nvidia-smi

The command should output more than one GPU. Inside a program, the number of GPU in the node can be obtained using the `cudaGetDeviceCount(int *count)` function and to perform any task, like running a CUDA kernel, copy operation, etc. on a particular GPU, we use the `cudaSetDevice(int device)` function.

### Copying between GPUs

The `cudaMemcpy` function supports GPU to GPU copy using the `cudaMemcpyDeviceToDevice` flag and the source and destination memory addresses should reside in GPU devices. 

For example, if we want to copy 1000 floats from the array `arr_gpu_0` allocated on GPU 0 to the array `arr_gpu_1`, the function call is:

```c
cudaMemcpy(arr_gpu_1, arr_gpu_0, 1000 * sizeof(float), cudaMemcpyDeviceToDevice);
```

Recall that CUDA kernel calls made from the host are non-blocking (asynchronous) by default. That is, the control may return back to the host thread before the device kernel finishes execution. To perform the halo exchange, we need to perform copy operations between each GPU and its neighbours. However, for large copy sizes, `cudaMemcpy` is blocking with respect to the host. 

Thus, we cannot use the following code snippet:

```c
for (int i = 0; i < 2; i++) {
    // Set current device
    cudaSetDevice(i);
    // Define row number of top and bottom neighbours, etc.
    TopNeighbour = ...; BotNeighbour = ...; // and so-on
    // Launch device kernel on GPU i
    jacobi_kernel<<<dim_grid, dim_block>>>(...);
    // Halo exchange
    cudaMemcpy(grid_rows[TopNeighbour], grid_rows[myTop], size, cudaMemcpyDeviceToDevice);
    cudaMemcpy(grid_rows[BotNeighbour], grid_rows[myBot], size, cudaMemcpyDeviceToDevice);
    // Norm check, swapping current and previous grid arrays, etc.
} // Serializes operations with respect to the host
```

As this code results in serialized execution:

![memcpy_serialized](../../images/memcpy_serialized.png)

### Asynchronous operations

Instead of `cudaMemcpy`, we can use the `cudaMemcpyAsync` function which is asynchronous with respect to the host. This allows the host to launch device kernels and copy operations concurrently, enabling parallel execution across GPUs. 

The correct code snippet is as follows:

```c
for (int i = 0; i < 2; i++) {
    // Set current device
    cudaSetDevice(i);
    // Launch device kernel on GPU i
    jacobi_kernel<<<dim_grid, dim_block>>>(...);
}
for (int i = 0; i < 2; i++) {
    // Define row number of top and bottom neighbours, etc.
    TopNeighbour = ...; BotNeighbour = ...; // and so-on
    // Halo exchange, notice the use of Async function
    cudaMemcpyAsync(grid_rows[TopNeighbour], grid_rows[myTop], size, cudaMemcpyDeviceToDevice);
    cudaMemcpyAsync(grid_rows[BotNeighbour], grid_rows[myBot], size, cudaMemcpyDeviceToDevice);
    // Norm check, swapping current and previous grid arrays, etc.
} // Parallel execution across multiple GPUs
```

And the execution time of the application is reduced:

![memcpyasync_parallel](../../images/memcpyasync_parallel.png)

## Implementation exercise: Part 1

Now, let's parallelize our code across multiple GPUs by using `cudaSetDevice` and `cudaMemcpyAsync` operations. Open the [jacobi_memcpy.cu](../../source_code/memcpy/jacobi_memcpy.cu) file by using the `File` $\rightarrow$ `Open...` option.

Understand the flow of the program from within the `main` function. Review the following pre-Jacobi-computation steps:

1. Computation of the memory chunk size to be allocated on each GPU stored in the `chunk_size` integer array.
2. Allocation of memory on each GPU: Notice the use of array pointers like `a_new`, `l2_norm_d`, `iy_start`, etc. that point to device arrays allocated on GPU pointed to by `dev_id` variable.
3. Initialization of Dirichlet boundary conditions on left and right boundaries.
4. Share of initial top and bottom local grid-point values between neighbours.


Now, within the iterative Jacobi loop (the `while` loop), implement the following marked as `TODO: Part 1-`:

1. Set current GPU and call device kernel with correct device arrays in function arguments.
2. Asynchronously copy GPU-local L2 norm back to CPU and implement top and bottom halo exchanges.
3. Synchronize the devices at the end of each iteration using `cudaDeviceSynchronize` function.

Review the topic on Asynchronous Operations above if in doubt. Recall the utility of using separate `for` loops for launching device kernels and initiating copy operations.

After implementing these, let's compile the code:

In [ ]:
!cd ../../source_code/memcpy && make clean && make jacobi_memcpy

Ensure there are no compiler warnings or errors. Validate the implementation by running the binary:

In [ ]:
!cd ../../source_code/memcpy && ./jacobi_memcpy

The last couple of lines of the output will give the number and IDs of GPUs used, execution timings, speedup, and efficiency metrics. Review Metrics of Interest section in [single GPU overview](../single_gpu/single_gpu_overview.ipynb) tutorial for more information). We tested the code on a DGX-1 system with 8 Tesla V100 16GB GPUs, and we got the following output:

```bash
Num GPUs: 8. Using GPU ID: 0, 1, 2, 3, 4, 5, 6, 7, 
16384x16384: 1 GPU:   5.0272 s, 8 GPUs:   1.1376 s, speedup:     4.42, efficiency:    55.24
```

Notice that we got a speed-up of $4.42\times$ using 8 GPUs and a corresponding efficiency of $55.24\%$. The numbers will vary depending on number of available GPUs in your system, the communication topology, GPU type, etc.

### Profiling

Now, profile the execution with `nsys`:

In [ ]:
!cd ../../source_code/memcpy/ && nsys profile --trace=cuda,nvtx --stats=true -o jacobi_memcpy_sys_report --force-overwrite true ./jacobi_memcpy -gpus 0,7

In the profiler timeline, the first few seconds denote the single-GPU code running on one of the GPUs. This version is executed so we can compare the multi-GPU version with it and we have already analyzed it. Let's analyze the multi-GPU timeline:

IMAGE LINK HERE

NSYS DESCRIPTION HERE

The solution for this exercise is present in `source_code/memcpy/solution` directory: [jacobi_memcpy.cu](../../source_code/memcpy/solution/jacobi_memcpy.cu)

## CUDA concepts: Part 2

### Host Staging of Copy Operations

Using `cudaMemcpyAsync` instead of `cudaMemcpy` allows us to issue copy and compute operations on multiple GPUs concurrently. The path taken by the data in both the cases is denoted by the red arrow as follows:

![memcpy_host_staging](../../images/memcpy_host_staging.png)

That is, in the GPU-to-GPU memory copy, the data traverses from GPU 0 the PCIe bus to the CPU, where it is staged in a buffer before being copied to GPU 1. This is called "host staging" and it decreases the bandwidth while increasing the latency of the operation. If we eliminate host staging, we can usually improve the performance of our application.

### Peer-to-Peer Memory Access

P2P allows devices to address each other's memory from within device kernels and eliminates host staging by transferring data either through the PCIe switch or through NVLink as denoted by the red arrow below. 

![memcpy_p2p_overview](../../images/memcpy_p2p_overview.png)

Peer-to-Peer (P2P) memory access requires GPUs to share a Unified Virtual Address Space (UVA). UVA means that a single address space is used for the host and all modern NVIDIA GPU devices (specifically, those with compute capibility of 2.0 or higher).

This P2P memory access feature is supported between two devices if `cudaDeviceCanAccessPeer()` returns true for these two devices. P2P must be enabled between two devices by calling `cudaDeviceEnablePeerAccess()` as illustrated in the following code sample:

```c
cudaSetDevice(currDevice);
int canAccessPeer = 0;
cudaDeviceCanAccessPeer(&canAccessPeer, currDevice, PeerDevice);
if (canAccessPeer) {
    cudaDeviceEnablePeerAccess(PeerDevice, 0);
}
```

Note that this enables a unidirectional P2P access where `currDevice` can perform memory access to `PeerDevice`. If we want `PeerDevice` to be able to access `currDevice` via P2P, then we need to use the code accordingly.

First, let's check if P2P is supported between the GPUs:

In [ ]:
!nvidia-smi topo -p2p r

The `topo` sub-command requests information on the GPU communication topology, `-p2p` flag requests P2P status, and `r` asks whether P2P reads are supported. Change `r` to `w` to check whether writes are supported. We share our output on a DGX-1 system with 8 Tesla V100s, focusing on the capabilities of GPU 0:

![nvidia_smi_p2p_gpu0](../../images/nvidia_smi_p2p_gpu0.png)

This means GPU 0 can communicate via P2P with GPUs 1 through 4. For GPUs 5 through 7, it must use host staging.

To check whether P2P via NVLink is supported, run the command below:

In [ ]:
!nvidia-smi topo -p2p n

In our DGX-1 system, the result is similar as before. Even if P2P via NVLink is not supported on your system, as long as `-p2p r` and `-p2p w` are supported between GPUs, P2P capability is available.

## Implementation Exercise: Part 2

Now, let us improve our program performance by enabling P2P access between GPUs, wherever possible. The `jacobi_memcpy.cu` code accepts a runtime argument `-p2p` which should enable P2P access between GPUs. 

Modify the code by searching for `TODO: Part 2` and enabling GPU `devices[dev_id]` to access peer GPUs `devices[top]` and `devices[bottom]`, whenever possible. 

Notice that the code snippet is within a `for` loop which sets and iterates over each GPU, which is why bidirectional P2P will be enabled. Take help from the code sample in the previous section.

Now, let's compile the code again:

In [ ]:
!cd ../../source_code/memcpy && make clean && make jacobi_memcpy

Ensure there are no compiler warnings or errors. Validate the implementation by running the binary with P2P enabled:

In [ ]:
!cd ../../source_code/memcpy && ./jacobi_memcpy -p2p

The output we got on our DGX-1 system is:

```bash
Num GPUs: 8. Using GPU ID: 0, 1, 2, 3, 4, 5, 6, 7, 
16384x16384: 1 GPU:   4.4487 s, 8 GPUs:   0.8798 s, speedup:     5.06, efficiency:    63.21 
```

Notice that the efficiency increased by about $8\%$ to $63.21\%$ compared to our baseline implementation. You can run the baseline again by removing the `-p2p` flag. Note that if P2P is not supported on your system, you will likely not experience any performance gain.

### Profiling

IMAGE LINK HERE

NSYS DESCRIPTION HERE

## Intra-Node Communication Topology

Run the command below to display your node's GPU and NIC communication topology:

In [ ]:
!nvidia-smi topo -m

If the output is unclear, you can launch a Terminal session by clicking on `File` $\rightarrow$ Open and following the steps as shown:

![open_terminal_session](../../images/open_terminal_session.png)

On our DGX-1 system, the output is as follows:

![nvidia_smi_topo_output](../../images/nvidia_smi_topo_output.png)

Focus one a particular row, say GPU 0. The output states that GPUs 1 through 4 are connected to it via NVLink (in addition to PCIe) and GPUs 5 through 7 are connected to it via PCIe as well as an "SMP" interconnect. We have a dual-socket system and the CPUs in these sockets are connected by an interconnect known as SMP interconnect.

Thus, GPU 0 to GPU 5 communication happens via not just PCIe, but also over the inter-socket interconnect within the same node. Clearly, this is a longer path than say the one between GPU 0 and GPU 1, which are connected via NVLink directly. We will discuss the NIC to GPU connection in the inter-node section of this bootcamp.

Even within the GPUs connected via NVLink, we see different annotations such as `NV1` and `NV2` that affect the communication bandwidth and hence the performance. In this section, we will explore the nuances associated with a diverse intra-node GPU communication topology like in the output above. Specifically, in our system, the communication topology is as follows:

![dgx1_8x_tesla_v100_topo](../../images/dgx1_8x_tesla_v100_topo.png)

Qualitatively, the bandwidth and latency vary with the topology as follows:

![intra_node_topology_map](../../images/intra_node_topology_map.png)

Host staging implies traversing through the CPU and the travel path taken is one of PHB, NODE, and SYS. In contrast, if the path taken is either NV1, NV2, or PIX, then P2P is available. PXB implies that the GPUs belong to different PCIe hubs and P2P is usually not supported in this case.

A double NVLink connection provides twice the bandwidth compared to a single NVLink. 

For a pair of 2 GPUs, the peak bidirectional bandwidth are as follows:
* PCIe: Using PIX topology, 15.75GB/s for PCIe Gen 3.0 and 31.5GB/s for PCIe Gen 4.0.
* NVLink: Using NV# topology, 50GB/s per connection. So a double NVLink connection has 100GB/s peak bidirectional bandwidth.

Let us understand what difference the underlying communication topology can make to the application performance in the following sub-section.

**Note:** If your command output doesn't show any NVLink connection or if there's no difference in connection type (PIX, PXB, PHB, NODE, SYS, NV#) between any 2 pair of GPUs, then the communication bandwidth and latency will likely be the same between any pair and the following sub-sections will not display any performance difference.

### Performance variation due to system topology

So far, the code runs the multi-GPU version on all available GPUs in a node (8 in our case). We can supply the `-gpus` runtime flag to the binary to run our code on specific GPUs. If we want to run on only 2 GPUs, namely GPU 0 and GPU 3, we use the `-gpus 0,3` argument. 

Try to find the GPU pair with highest bandwidth available as per the table above and replace `0,3` with those GPUs, and then run the command below:

In [ ]:
!cd ../../source_code/memcpy && ./jacobi_memcpy -p2p -gpus 0,7

The efficiency would likely be higher than before due to less inter-GPU communication (each GPU does more wok instead). Our output is as follows:

```bash
Num GPUs: 2. Using GPU ID: 0, 3, 
16384x16384: 1 GPU:   4.4513 s, 2 GPUs:   2.2664 s, speedup:     1.96, efficiency:    98.20  
```

Now, run the binary a pair of GPUs that have the lowest available bandwidth. In our case, we use GPU 0 and GPU 7. Our output is:

```bash
Num GPUs: 2. Using GPU ID: 0, 7, 
16384x16384: 1 GPU:   4.4529 s, 2 GPUs:   2.3454 s, speedup:     1.90, efficiency:    94.93  
```

Now remove the `-p2p` flag and run the command again for GPUs 0 and 7. We didn't get any difference in performance. As you may recall, P2P is not possible between GPUs 0 and 7, so the underlying communication path doesn't change, resulting in same performance with and without the `-p2p` flag. The same can be confirmed by profiling the application and looking at the operations performed in the Nsight Systems timeline. 

![p2p_2_gpu_memcpy_nsys](../../images/p2p_2_gpu_memcpy_nsys.png)

Try a few other GPU combinations and toggle P2P so see if the performance variation correlates with the table above. Also try reducing the grid size using `-nx` and `-ny` flags (to say 8192$\times$8192) and see the effect on efficiency. 

### Benchmarking the system topology

Our application is not very memory intensive. As is visible from the profiler output, $\gt95\%$ of the time in GPU is spent on computation. Therefore, to get a quantitative measure of latency and bandwidth impact due to topology, we run a micro-benchmark.

**The p2pBandwidthLatencyTest micro-benchmark**

p2pBandwidthLatencyTest is a part of [CUDA Samples GitHub repository](https://github.com/NVIDIA/cuda-samples) available to help CUDA developers. 

As the name suggests, this test measures the bandwidth and latency impact of P2P and underlying communication topology. Let's compile the benchmark:

In [ ]:
!cd ../../source_code/p2pBandwidthLatencyTest/ && make clean && make

Now, let's run the benchmark:

In [ ]:
!cd ../../source_code/p2pBandwidthLatencyTest/ && ./p2pBandwidthLatencyTest

The first part of the benchmark gives device information and P2P access available from each GPU (similar to `nvidia-smi topo -m` command). Next, the benchmark measures the unidirectional and bidirectional bandwidth and latency with P2P disabled and enabled.

We share partial results obtained in our DGX-1 system:

```bash
Bidirectional P2P=Disabled Bandwidth Matrix (GB/s)
   D\D     0      1      2      3      4      5      6      7 
     0 783.95   9.56  14.43  14.46  14.47  14.24  14.51  14.43 

Bidirectional P2P=Enabled Bandwidth Matrix (GB/s)
   D\D     0      1      2      3      4      5      6      7 
     0 784.87  48.49  48.49  96.85  96.90  14.25  14.54  14.49 
     
P2P=Disabled Latency Matrix (us)
   GPU     0      1      2      3      4      5      6      7 
     0   1.78  17.52  16.41  16.43  17.35  16.88  17.34  16.85 
     
P2P=Enabled Latency (P2P Writes) Matrix (us)
   GPU     0      1      2      3      4      5      6      7 
     0   1.76   1.62   1.61   2.01   2.02  18.44  19.15  19.34
```

Our system is based on PCIe gen 3.0 with a peak maximum GPU-GPU PCIe banwidth of 15.75 GB/s. Let us analyze and understand these results:

* GPU 0 and GPU 1/2: Connected by a single NVLink connection. By enabling P2P-
  - Bandwidth reaches close to the maximum peak of 50 GB/s.
  - Latency decreases by an order of magnitude.
* GPU 0 and GPU 3/4: Connected by a double NVLink connection. By enabling P2P-
  - Bandwidth reaches close to the maximum peak of 100 GB/s.
  - Latency decreases by an order of magnitude.
* GPU 0 and GPU 5/6/7: Connected by PCIe and SMP interconnect. By enabling P2P- 
  - Bandwidth is unchanged.
  - Latency increases a marginally.
  
Correlate these results with the communication topology that can be displayed by usng `nvidia-smi topo -m` command and the qualtitative table in the previous section. They should be consistent with one another.

In general, we should try to set the GPUs in an application such that a GPU can share data with its neighbours using a high-bandwidth, low-latency communication topology. Enabling P2P, when possible, usually improves the performance by eliminating host staging.